In [ ]:
# 加载环境变量
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

#### 使用LangChain调用文本类模型,以OpenAI为例

In [ ]:
#使用openai的官方sdk
from openai import OpenAI
import os

client = OpenAI(
    # api_base=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    
)

messages = [
    {"role": "user", "content": "介绍下你自己"}
]

res = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=message,
    stream=False,
)

print(res['choices'][0]['message']['content'])

#### 使用LangChain调用对话类模型,以ChatOpenAI为例

In [ ]:
#调用对话类，以openai为例

from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,AIMessage
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

chat = ChatOpenAI(
    base_url=api_base,
    api_key=api_key,
    model="gpt-4",
    temperature=0,
)

messages = [
    AIMessage(role="system",content="你好，我是tomie！"),
    HumanMessage(role="user",content="你好tomie，我是狗剩!"),
    AIMessage(role="system",content="认识你很高兴!"),
    HumanMessage(role="user",content="你知道我叫什么吗？")
]

response = chat.invoke(messages)
print(response)

#### langchain内置的LLM支持情况

![LLMs](images/LLMs.png)

#### LangChain调用大模型流式输出

In [ ]:
# LLM文本类大模型的流式输出方法

from langchain_community.llms import OpenAI
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

# 构造一个llm实例
llm = OpenAI(
    # openai_proxy=api_base,
    openai_api_key=api_key,
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    
    max_tokens=512,
)

# 流式输出
for chunk in llm.stream("写一首关于秋天的诗歌"):
    # flush:缓冲区刷新禁用
    print(chunk, end="", flush=False)

In [ ]:
%pip install anthropic

In [ ]:
# Chat Models的流式调用方法
# 使用clade模型

from langchain_community.chat_models import ChatOpenAI
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

# 构建一个ChatOpenAI实例
llm = ChatOpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model = "claude-3-opus-20240229",
    temperature=0,
    max_tokens=512,
)

# 流式输出
for chunk in llm.stream("写一首关于秋天的诗歌"):
    print(chunk,end="\n",flush=False)

### 追踪Token的使用

In [ ]:
# LLM文本类模型的token追踪
from langchain_community.llms import OpenAI
from langchain_community.callbacks import get_openai_callback
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

#构造一个llm
llm = OpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    max_tokens=512,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲一个笑话")
    print(result)
    # 打印token使用情况
    print(cb)


In [ ]:
# Chat Models的token追踪
from langchain_community.chat_models import ChatOpenAI
from langchain_community.callbacks import get_openai_callback
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

llm = ChatOpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model = "gpt-4",
    temperature=0,
    max_tokens=512,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲一个笑话")
    print(result)
    print(cb)

### 自定义输出

- 输出函数参数
- 输出json
- 输出List
- 输出日期


In [ ]:
#讲笑话机器人：希望每次根据指令，可以输出一个这样的笑话(小明是怎么死的？笨死的)

from langchain_community.llms import OpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel,Field,validator
from typing import List
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

#构造LLM
model = OpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model="gpt-3.5-turbo-instruct",
    temperature=0,
)

#  定义个数据模型，用来描述最终的实例结构
class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str = Field(description="回答笑话的答案")

    # 验证问题是否符合要求
    @validator("setup")
    def question_mark(cls,field):
        if field[-1] != "？":
            raise ValueError("不符合预期的问题格式!")
        return field

# 将Joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)


prompt = PromptTemplate(
    template="回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

# 管道符调用,将prompt和model进行组合,prompt的结果作为model的输入
prompt_and_model = prompt | model
out_put = prompt_and_model.invoke({"query":"给我讲一个笑话"})
print("out_put:", out_put)
# 调用parser对输出进行解析
parser.invoke(out_put)

In [ ]:
# LLM的输出格式化成python list形式，类似['a','b','c']

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

# 构造LLM
model = OpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model="gpt-3.5-turbo-instruct",
    temperature=0,
)

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="列出5个{subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

_input = prompt.format(subject="常见的小狗的名字")
output = model(_input)
print(output)
# 调用parser对输出进行解析
parser.parse(output)